In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.metrics import RocCurveDisplay


In [16]:
# Initialize functions
scale = StandardScaler()


In [14]:
# Load data with regional brain volumes and covariates
df = pd.read_csv('data.csv')
# Print summary of data
df.head()

,SubjID_FreeSurfer,ID,IDs_vol,site,site_num,CNV,genetic_stat2,age_spm,sex,volume,...,I-IV_Cerebellum,V_Cerebellum,VI_Cerebellum,Crus_I_Cerebellum,Crus_II_Cerebellum,VIIb_Cerebellum,VIIIa_Cerebellum,VIIIb_Cerebellum,IX_Cerebellum,X_Cerebellum
570,1052340,1052340,1052340,6.0,6.0,15q11.2,deletion,55.0,0.0,587914.0,...,-0.146308,-0.053500,0.016681,0.020838,0.046743,-0.067465,-0.027757,0.045679,-0.104263,0.076513
571,1066044,1066044,1066044,6.0,6.0,15q11.2,deletion,48.0,0.0,592574.0,...,0.007876,0.100267,0.021185,0.047022,0.093325,-0.146724,0.019252,-0.100824,-0.082475,-0.039452
572,1108629,1108629,1108629,6.0,6.0,15q11.2,deletion,66.0,1.0,611745.0,...,-0.125297,0.013399,0.095192,-0.103760,0.159400,0.082095,0.079173,0.462184,-0.148444,0.159370
573,1115864,1115864,1115864,7.0,7.0,15q11.2,deletion,44.0,1.0,627202.0,...,-0.139433,0.006227,-0.042850,0.082768,-0.008729,-0.040657,0.112364,0.180582,-0.065167,0.395848
574,1128201,1128201,1128201,6.0,6.0,15q11.2,deletion,60.0,1.0,602985.0,...,-0.283510,-0.072724,0.042752,-0.047456,-0.044656,-0.008032,0.014380,0.071450,-0.052110,-0.029071


Create dataframes for CNV carriers and controls.

In [18]:
df_mutsubs = df[df.CNV != 'Control']
df_controls = df[df.CNV == 'Control']

# Arrays of regional volumes
X_full = np.array(df.iloc[:, 10:])
X_ctrl = X_full[df.CNV == 'Control']
X_mut = X_full[df.CNV != 'Control']

Define functions for bootstrapping.

In [19]:
def optimal_model(X, y, X_coh, cat_id):
    lda = LinearDiscriminantAnalysis(solver='svd')
    lda.fit(X, y)
    # re-order discriminant function
    if np.corrcoef(lda.scalings_[:, 0], X_coh[cat_id, :])[0, 1] > 0:
        lda.scalings_ = -lda.scalings_
    return lda


def create_bset(X, y, boot):
    X_ctrl = X[y == 0]
    X_mut = X[y == 1]
    n_ctrl = len(X_ctrl)
    n_mut = len(X_mut)
    id_ctrl = np.arange(0, n_ctrl)
    id_mut = np.arange(0, n_mut)
    id_ctrl_train = resample(id_ctrl, replace=True, n_samples=n_ctrl,
                             random_state=boot)
    id_mut_train = resample(id_mut, replace=True, n_samples=n_mut,
                            random_state=boot)
    X_boot = np.concatenate((X_mut[id_mut_train, :],
                             X_ctrl[id_ctrl_train, :]))
    return X_boot

Define classification targets.

In [20]:
category_names = ['1q21.1del', '1q21.1dup', '16p11.2del', '16p11.2dup',
                  '22q11.2del', '22q11.2dup']
target_mut = df_mutsubs.CNV.map({'1q21.1': 0, '15q11.2': 1, '16p11.2': 2,
                                 '22q11.2': 3})
y_mut = df_mutsubs.genetic_stat2.map({'deletion': 0, 'duplication': 1})
y = np.array(y_mut+(target_mut*2))

n_boot = 1000  # nb of bstraps

Extract asymmetry patterns using bootstrapping.

In [ ]:
for i, category in enumerate(category_names):
    print(category)

    # Select CNV and controls
    tmp_X = X_mut[y == i, :]
    X_star = np.concatenate((tmp_X, X_ctrl))

    # Scale data
    X_star_ss = scale.fit_transform(X_star)

    # Classification targets
    tmp_y = y[y == i]

    y_star = np.concatenate((tmp_y, np.ones(X_ctrl.shape[0])*10))
    y_star = (y_star < 10)*1
    models = []
    for j in range(n_boot):
        X_boot = create_bset(X_star_ss, y_star, j)
        model = optimal_model(X_boot, y, X_coh, i)
